# Principios y Patrones del Desarrollo de Software.

### **Pregunta 1. SOLID.**

Explique en detalle el principio SOLID "Open/Closed" y proporcione un ejemplo de código en Python donde este principio se ha violado y cómo puede corregirlo.

Primero de todo, SOLID se trata de un acrónimo que hace referencia a los cinco principios básicos de diseño con el fin de hacer que los diseños de software sean más comprensibles.

"Open/Closed" se trata del segundo principio SOLID. Se refiere a que los módulos de software deben de ser **abiertos para su extensión, pero cerrados para su modificación**. Eso se refiere a que primero el comportamiento del software pueda extenderse. Es decir, si se cambian los requisitos, que se pueda ampliar el módulo para adaptarlo a estos. A su vez, con que el módulo este cerrado nos referimos a que si extendemos el comportamiento, los demás módulos no deben de verse afectamos por el cambio ni modificarse.

#### **Open/Closed - Modificar.**
Supongamos que tenemos una clase compra que tiene una lista de productos y sus precios. Si queremos añadir tipos de **descuentos** tendríamos que modificar la clase todo el rato.

Esto viola el principio de Open/Closed porque estamos cambiando una clase ya existente para añadir una nueva funcionalidad.

In [30]:
class Compra():
    def __init__(self):
        self.productos = []
    def add_product(self, producto, precio):
        self.productos.append([producto, precio])
    def total(self):
        total = 0
        for producto in self.productos:
            total += producto[1]
        return total
    def descuento(self, tipo_descuento):
        if tipo_descuento == "navidad":
            return self.total() * 0.80
        if tipo_descuento == "fin de mes":
            return self.total() * 0.90
        if tipo_descuento == "Black Friday":
            return self.total() * 0.50
        # y así con otros descuentos...

#### **Open/Closed - Modificado.**
Para cumplir con el principio Open/Closed se debe de aplicar el **polimorfismo**. El polimorfismo hace que diferentes objetos pueden responder de manera distinta al mismo mensaje o método. Con esto permitiríamos la agregación de distintos tipos de descuento sin necesidad de modificar la clase ya creada: `Compra`.

En lugar de modificar la clase, se creará una clase base Descuento, siendo esta abstracta ya que no implementa completamente los métodos que declara, sino que delega la implementación específica a las subclases; y luego extenderla con diferentes tipos de descuentos sin modificar la clase original.

In [31]:
from abc import ABC, abstractmethod

In [32]:
class Descuento(ABC):
    @abstractmethod
    def apply(self, compra):
        pass

class NavidadDescuento(Descuento):
    def apply(self, compra):
        return compra * 0.80

class FinDeMesDescuento(Descuento):
    def apply(self, compra):
        return compra * 0.90

class BlackFridayDescuento(Descuento):
    def apply(self, compra):
        return compra * 0.50

In [33]:
class Compra():
    def __init__(self):
        self.productos = []
    def add_product(self, producto, precio):
        self.productos.append([producto, precio])
    def total(self):
        total = 0
        for producto in self.productos:
            total += producto[1]
        return total
    def descuento(self, tipo_descuento: Descuento):
        return tipo_descuento.apply(self.total())

    def __str__(self):
        return f"Compra: {self.productos}, Total: {self.total()}"

In [34]:
if __name__ == "__main__":
    compra = Compra()
    descuentos = [BlackFridayDescuento(), NavidadDescuento(), FinDeMesDescuento()]
    compra.add_product("Camisa", 50)
    compra.add_product("Pantalón", 50)
    print(compra)
    for i in descuentos:
        print(i.__class__.__name__, ': ',compra.descuento(i))

Compra: [['Camisa', 50], ['Pantalón', 50]], Total: 100
BlackFridayDescuento :  50.0
NavidadDescuento :  80.0
FinDeMesDescuento :  90.0


In [35]:
if __name__ == "__main__":
    compra.add_product("Zapas", 100)
    print(compra)
    for i in descuentos:
        print(i.__class__.__name__, ': ',compra.descuento(i))

Compra: [['Camisa', 50], ['Pantalón', 50], ['Zapas', 100]], Total: 200
BlackFridayDescuento :  100.0
NavidadDescuento :  160.0
FinDeMesDescuento :  180.0


## **Pregunta 2. FACTORY.**
Describa el patrón de diseño "Factory". ¿En qué situaciones sería útil este patrón? Proporcione un ejemplo de cómo implementaría este patrón en Python para un problema relacionado con la ingeniería matemática, como la creación de diferentes tipos de funciones matemáticas.

**Factory** se trata de un patrón creacional. Los patrones creacionales ofrecen formas de generar instancias de clases, mejorando la adaptabilidad y la reutilización del código.

**Factory** se trata de un patrón que provee una clase abstracta, `Creator`, que permite encapsular la lógica de creación de los objetos en subclases y estas deciden que clase instanciar. Estos objetos se crean a patir de un método (NO CONSTRUCTORES). Sigue la siguiente estructura:
- Product: Definición de las interfaces para la familia de productos genéricos.
- ConcreteProduct: Implementación de los diferentes productos.
- Creator: Declara el método encargado de instanciar nuevos objetos. Suele ser una clase abstracta. Clase `Factory`.
- ConcreteCreator: Crea la instancia del producto concreto.

**Utilidades del Patrón Factory.**

El patrón Factory resulta especialmente útil cuando **no se conoce de antemano el tipo exacto de objeto a crear** y es necesario generarlo en tiempo de ejecución. Permite incorporar nuevas subclases de `Product` sin modificar el código cliente, lo que incrementa la **adaptabilidad** del sistema. Además, es ideal en escenarios donde es importante **centralizar la lógica de creación de objetos en un único punto**. Finalmente, el patrón es adecuado cuando el tipo de objeto a crear puede variar en función de determinados parámetros o condiciones, manteniendo el código limpio y flexible.


In [36]:
# CLASE Product: Función genérica que declara las operaciones que todas las funciones matemáticas tienen en común.
class MathFuncion(ABC):
    @abstractmethod
    def calcular(self, x):
        pass

In [37]:
# CLASE ConcreteProduct: Diferentes productos.
class FuncionLineal(MathFuncion):
    def __init__(self, a, b):
        self.a = a
        self.b = b
    def calcular(self, x):
        return self.a * x + self.b
    def operacion(self):
        return f'f(x) = {self.a}x + {self.b}'

class FuncionCuadratica(MathFuncion):
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c
    def calcular(self, x):
        return self.a * x**2 + self.b * x + self.c
    def operacion(self):
        return f'f(x) = {self.a}x^2 + {self.b}x + {self.c}'

In [38]:
# CLASE Creator: Clase abstracta que declara el método de fabrica que devolverá una función.
class FuncionCreator(ABC):
    @abstractmethod
    def factory_method(self, tipo_funcion, *args) -> MathFuncion:
        pass
    def calcular(self, x, tipo_funcion, *args): # número, tipo función, coeficientes.
        funcion = self.factory_method(tipo_funcion, *args)
        return f"FunctionCreator: Calculando usando {funcion.calcular(x)} en {funcion.operacion()}"

In [39]:
# CLASE ConcreteCreator: Implementa el método de fábrica que devuelve un objeto de la clase concreta.
class ConcreteFuncionCreator(FuncionCreator):
    def factory_method(self, tipo_funcion, *args) -> MathFuncion:
        if tipo_funcion == "lineal":
            a = args[0]
            b = args[1]
            return FuncionLineal(a, b)
        if tipo_funcion == "cuadratica":
            a = args[0]
            b = args[1]
            c = args[2]
            return FuncionCuadratica(a, b, c)
        else:
            return 'Tipo de función desconocida'


In [40]:
# Código del cliente: Instancia de un creator concreto sin conocer su clase específica.
def client_code(creator: FuncionCreator, x, function_type, *args):
    print(creator.calcular(x, function_type, *args))

In [41]:
if __name__ == "__main__":
    creator = ConcreteFuncionCreator()
    
    # Ejemplo usando el creador para diferentes tipos de funciones
    print("App: Lanzado con funciones lineales.")
    client_code(creator, 5, "lineal", 2, 3)  # f(x) = 2*x + 3
    
    print("\nApp: Lanzado con funciones cuadráticas.")
    client_code(creator, 5, "cuadratica", 1, -2, 1)  # f(x) = 1*x^2 - 2*x + 1

App: Lanzado con funciones lineales.
FunctionCreator: Calculando usando 13 en f(x) = 2x + 3

App: Lanzado con funciones cuadráticas.
FunctionCreator: Calculando usando 16 en f(x) = 1x^2 + -2x + 1


## **Pregunta 3. God Object.**